In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial as sr
import pandas as pd
import time

# 아두이노로 전송하기 위한 포트 지정
# 포트번호 맞게 변경
ard=sr.Serial('/dev/cu.usbmodem11301',9600)       # 전광판  sd카드칸 바로옆        

max_num_hands = 1

# 전광판
gesture = {
    0: 'off', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five',
    6: 'ddabong', 7: 'call', 8: 'ok'
}
rps_gesture = {
    0: 'off', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five',
    6: 'ddabong', 7: 'call', 8: 'ok'
}

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Gesture recognition model
file = np.genfromtxt('data/gesture_train_led.csv', delimiter=',')
angle = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)
knn = cv2.ml.KNearest_create()
knn.train(angle, cv2.ml.ROW_SAMPLE, label)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        continue

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        rps_result = []

        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            # Inference gesture
            data = np.array([angle], dtype=np.float32)
            ret, results, neighbours, dist = knn.findNearest(data, 3)
            idx = int(results[0][0])

            # Draw gesture result
            if idx in rps_gesture.keys():
                org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0]))
                cv2.putText(img, text=rps_gesture[idx].upper(), org=(org[0], org[1] + 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
                
                # 모션인식한 값을 변수로 저장
                if idx == 0:
                    sign = '0'
                elif idx == 1:
                    sign = '1'
                elif idx == 2:
                     sign = '2'
                elif idx == 3:
                    sign = '3'
                elif idx == 4:
                    sign = '4'
                elif idx == 5:
                    sign = '5'
                elif idx == 6:
                    sign = '6'
                elif idx == 7:
                    sign = '7'
                elif idx == 8:
                    sign = '8'

                    
                # 아두이노로 값 전송
                ard.write(sign.encode())           # 전광판 아두이노로 전송
                
                
                rps_result.append({
                    'rps': rps_gesture[idx],
                    'org': org
                })

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

    cv2.imshow('camera', img)
    if cv2.waitKey(1) == ord('q'):
        break


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
